# Day 3 - Lab 2: Refactoring & Documentation

**Objective:** Use an LLM to refactor a complex Python function to improve its readability and maintainability, and then generate comprehensive, high-quality documentation for the project.

**Estimated Time:** 60 minutes

**Introduction:**
Writing code is only the first step; writing *good* code is what makes a project successful in the long run. In this lab, you will use an LLM as a code quality expert. You will refactor a poorly written function to make it cleaner and then generate professional-grade documentation, including docstrings and a README file. These are high-value tasks that AI can significantly accelerate.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will set up our environment and define a sample of poorly written code that we will use as the target for our refactoring and documentation efforts.

**Model Selection:**
Models with strong coding and reasoning abilities are best for this task. `gpt-4.1`, `o3`, or `codex-mini` are great choices. You can also try more general models like `gemini-2.5-pro`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `save_artifact()`: To save the generated README file.
- `clean_llm_output()`: To clean up the generated code and documentation.

In [10]:
import sys
import os
from importlib import reload

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, prompt_enhancer, load_artifact
import utils.llm as utils_llm

reload(utils_llm)
clean_llm_output = utils_llm.clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

2025-10-30 15:05:21,306 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Code to Improve

Here is a sample Python function that is functional but poorly written. It's hard to read, has no comments or type hints, and mixes multiple responsibilities. This is the code we will improve.

In [2]:
bad_code = """
def process_data(data, operation):
    if operation == 'sum':
        total = 0
        for i in data:
            total += i
        return total
    elif operation == 'average':
        total = 0
        for i in data:
            total += i
        return total / len(data)
    elif operation == 'max':
        max_val = data[0]
        for i in data:
            if i > max_val:
                max_val = i
        return max_val
"""

## Step 3: The Challenges

### Challenge 1 (Foundational): Refactoring the Code

**Task:** Use the LLM to refactor the `bad_code` to be more readable, efficient, and maintainable.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a senior Python developer.
2.  Provide the `bad_code` as context.
3.  Ask the LLM to refactor the code. Be specific about the improvements you want, such as:
    * Breaking the single function into multiple, smaller functions.
    * Using built-in Python functions where appropriate (e.g., `sum()`, `max()`).
    * Adding clear type hints and return types.

> **Tip:** When you ask the AI to refactor, give it a principle to follow. For example, ask it to apply the 'Single Responsibility Principle,' which means each function should do only one thing. This guides the AI to create cleaner, more modular code.

**Expected Quality:** A block of Python code that is functionally identical to the original but is significantly cleaner, more modular, and easier to understand.

In [6]:
# TODO: Write a prompt to refactor the 'bad_code'.
refactor_prompt = f"""
Refactor the code {bad_code} according to best python practices, improving readability, efficiency, and maintainability.
Utilize DRY principles, meaningful variable names, and modular functions.
Provide the refactored code only, without any explanations or comments.
"""
enhanced_refactor_prompt = prompt_enhancer(refactor_prompt)

print("--- Refactoring Code ---")
refactored_code = get_completion(enhanced_refactor_prompt, client, model_name, api_provider)
cleaned_code = clean_llm_output(refactored_code, language='python')
print(cleaned_code)

2025-10-30 15:01:03,984 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Refactoring Code ---
from typing import Callable, Dict, List, Union


def process_data(
    data: List[Union[int, float]], operation: str
) -> Union[int, float]:
    """
    Performs a specified statistical operation on a list of numbers.

    This refactored implementation uses a dispatch dictionary to map operation
    names to their corresponding functions, improving maintainability and
    extensibility. It leverages optimized built-in functions and handles
    edge cases gracefully by raising appropriate errors.

    Args:
        data: A list of integers or floats to be processed.
        operation: The name of the operation to perform ('sum', 'average', 'max').

    Returns:
        The result of the calculation as an integer or float.

    Raises:
        ValueError: If the operation is unknown or mathematically undefined
                    for the given data (e.g., average of an empty list).
    """
    def _calculate_average(numbers: List[Union[int, float]]) -> float:
  

### Challenge 2 (Intermediate): Generating Docstrings

**Task:** Prompt the LLM to generate high-quality docstrings for the newly refactored code.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `refactored_code` from the previous step as context.
3.  Instruct the LLM to generate Google-style Python docstrings for each function.
4.  The docstrings should include a description of the function, its arguments (`Args:`), and what it returns (`Returns:`).

**Expected Quality:** The refactored Python code, now with complete and professional-looking docstrings for each function.

In [7]:
# TODO: Write a prompt to add Google-style docstrings to the refactored code.
docstring_prompt = f"""
Generate Google-style docstrings for the following Python code, ensuring clarity and completeness:{cleaned_code}
"""

print("--- Generating Docstrings ---")
code_with_docstrings = get_completion(docstring_prompt, client, model_name, api_provider)
cleaned_code_with_docstrings = clean_llm_output(code_with_docstrings, language='python')
print(cleaned_code_with_docstrings)

--- Generating Docstrings ---
from typing import Callable, Dict, List, Union


def process_data(
    data: List[Union[int, float]], operation: str
) -> Union[int, float]:
    """Performs a specified statistical operation on a list of numbers.

    This function uses a dispatch dictionary to map operation names to their
    corresponding implementations (e.g., 'sum' -> `sum()`). This approach
    avoids complex conditional logic and makes the function easily extensible.
    It handles edge cases, such as operating on an empty list, by raising
    a `ValueError`.

    Args:
        data (List[Union[int, float]]): A list of numbers (integers or floats)
            to be processed.
        operation (str): The name of the operation to perform. Valid options
            are 'sum', 'average', and 'max'.

    Returns:
        Union[int, float]: The result of the calculation. The type will be
        an integer or float depending on the operation and input data.

    Raises:
        ValueError

### Challenge 3 (Advanced): Generating a Project README

**Task:** Generate a comprehensive `README.md` file for the entire Onboarding Tool project.

**Instructions:**
1.  Create a final prompt that instructs the LLM to act as a technical writer.
2.  This time, you will provide multiple pieces of context: the `day1_prd.md` and the `app/main.py` source code. (You will need to load these files).
3.  Ask the LLM to generate a `README.md` file with the following sections:
    * Project Title
    * Overview (based on the PRD)
    * Features
    * API Endpoints (with `curl` examples)
    * Setup and Installation instructions.
4.  Save the final output to `README.md` in the project's root directory.

**Expected Quality:** A complete, professional `README.md` file that provides a comprehensive overview of the project for other developers.

In [18]:
# Load the necessary context files
prd_content = load_artifact("artifacts/day1_prd.md")
api_code = load_artifact("app/main.py")

# TODO: Write a prompt to generate a complete README.md file.
readme_prompt = f"""
Generate a comprehensive README.md file for a project based on the following Product Requirements Document (PRD) and API code.
Incorporate the project's purpose, features, installation instructions, usage examples, API endpoints, and any other relevant information.
Ensure the README is well-structured, clear, and engaging for potential users and developers.
{prd_content}
{api_code}
"""
enhanced_readme_prompt = prompt_enhancer(readme_prompt)
print(enhanced_readme_prompt)

print("--- Generating Project README ---")
if prd_content and api_code:
    readme_content = get_completion(enhanced_readme_prompt, client, model_name, api_provider)
    print("--- README Content ---")
    print(readme_content)

    
    cleaned_readme = clean_llm_output(readme_content, language='markdown')
    # print("--- Final README.md Content ---")
    # print(cleaned_readme)
    save_artifact(readme_content, "README.md", overwrite=True)
else:
    print("Skipping README generation because PRD or API code is missing.")

2025-10-30 15:30:39,636 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<prompt>
    <persona>
        You are a Senior Technical Writer and Developer Advocate with 10+ years of experience creating developer-focused documentation for SaaS platforms. You have deep knowledge of HRTech, FastAPI, PostgreSQL, and REST-first API design.
    </persona>

    <context>
        BACKGROUND:
        ──────────
        You are writing the primary README.md for “OnboardPro”, an employee-onboarding SaaS product.  
        The README must serve BOTH potential users (HR, hiring managers, new hires) and developers who may want to contribute or self-host.

        SOURCE MATERIALS TO USE VERBATIM WHEN RELEVANT:
        ------------------------------------------------
        1. Full Product Requirements Document (PRD) – see block <<PRD>>  
        2. Initial FastAPI code – see block <<API_CODE>>

        TOKEN BUDGET NOTE:
        You may freely quote or paraphrase the PRD and code, but keep the README concise (≈ 300–500 lines).  
        Link back to the full PRD instead of

## Lab Conclusion

Well done! You have used an LLM to perform two of the most valuable code quality tasks: refactoring and documentation. You've seen how AI can help transform messy code into a clean, maintainable structure and how it can generate comprehensive documentation from high-level project artifacts and source code. These skills are a massive productivity multiplier for any development team.

> **Key Takeaway:** LLMs excel at understanding and generating structured text, whether that structure is code or documentation. Providing a clear 'before' state (the bad code) and a clear goal (the refactoring principles) allows the AI to perform complex code transformation and documentation tasks efficiently.